## Step 1: Get YOLO Ready

In [ ]:
# clone YOLOv5 repo
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

In [ ]:
# clone YOLOv6 repo
!git clone https://github.com/meituan/YOLOv6
%cd YOLOv6

In [ ]:
# clone YOLOv7 repo
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7

In [ ]:
# for YOLO v5 v6 v7
# install required dependencies
%pip install -qr requirements.txt

# import required libraries
import torch

# check pytorch version, CUDA version and GPU model
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
# for YOLOv8 only
%pip install ultralytics
import ultralytics
ultralytics.checks()

## Step 2: Download Dataset

In [ ]:
# YOLOv5 YOLOv7 YOLOv8 have the same dataset structure
# install and import roboflow
%pip install -q roboflow
from roboflow import Roboflow
import os

# set up environment
os.environ["DATASET_DIRECTORY"] = "../datasets"

# download dataset from roboflow for specific yolo version
rf = Roboflow(api_key="VjyizEwlgGPXFO7JMvyw")
project = rf.workspace("forest-fire").project("second-annotation-for-trees")
dataset = project.version(8).download("yolov5")

In [ ]:
# For YOLOv6 only, run above code first, then run this code
%cd ../datasets/Second-Annotation-For-Trees-8/
%mkdir images
%mkdir labels
%mv test/labels labels/test
%mv train/labels labels/train
%mv valid/labels labels/valid
%mv test/images images/test
%mv train/images images/train
%mv valid/images images/valid
%cd ../../YOLOv6/

# For YOLOv6 only, finally, adjust data.yaml file (at ../datasets/Second-Annotation-For-Trees-8) as followed:

# nc: 2
# names: ['Alive Tree', 'Dead Tree']
# train: ../datasets/Second-Annotation-For-Trees-8/images/train
# val: ../datasets/Second-Annotation-For-Trees-8/images/valid
# test: ../datasets/Second-Annotation-For-Trees-8/images/test

## Step 3: Train YOLO Model with Dataset

In [ ]:
# train model YOLOv5
!python train.py --img 640 --batch 32 --epochs 3 --data ../datasets/Second-Annotation-For-Trees-8/data.yaml --weights yolov5s.pt --cache

In [ ]:
# train model YOLOv6
!python tools/train.py --img 640 --batch 32 --epochs 3 --data ../datasets/Second-Annotation-For-Trees-8/data.yaml --conf configs/yolov6s_finetune.py

In [ ]:
# train model YOLOv7
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt
!python train.py --workers 2 --device 0 --batch-size 32 --epochs 1 --data ../datasets/Second-Annotation-For-Trees-8/data.yaml --img 640 640 --weights 'yolov7.pt' --name yolov7-custom --hyp data/hyp.scratch.custom.yaml

In [ ]:
# train model YOLOv8
!yolo train model=yolov8n.pt data='../datasets/Second-Annotation-For-Trees-8/data.yaml' epochs=3 imgsz=640

In [ ]:
# Tensorboard (optional)
%reload_ext tensorboard
%tensorboard --logdir runs/train

## Step 4: Predict on Test Dataset

In [ ]:
# test model YOLOv5
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source ../datasets/Second-Annotation-For-Trees-8/test/images 

In [ ]:
# test model YOLOv6
!python tools/infer.py --weights runs/train/exp1/weights/best_ckpt.pt --img 640 --conf 0.25 --source ../datasets/Second-Annotation-For-Trees-8/images/test

In [ ]:
# test model YOLOv7
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source ../datasets/Second-Annotation-For-Trees-8/test/images 

In [ ]:
# test model YOLOv8
from ultralytics import YOLO
# Load a model
model = YOLO("yolov8n.pt")  # load an official model
model = YOLO("runs/detect/train/weights/best.pt")  # load a custom model

# Predict with the model
results = model.predict(source="../datasets/Second-Annotation-For-Trees-8/test/images", conf=0.25, save=True)  # predict on an image

In [ ]:
# download your model weights from some/path/to/best.pt

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>